# Data Standardisation

Combining AED and data Wrangling, at the end the csv produced should be a standardised dataset ready for use.

In [17]:
# Import libraries
import sys
import os
import pandas as pd
from pathlib import Path


sys.path.append("../")
os.getcwd()
from core.data_manager import csv_to_dataframe

In [18]:
# Variables
path = Path.cwd().parents[0]

path_water = path.joinpath("data_storage", "water.csv")

path_power = path.joinpath("data_storage", "power.csv")
path_gas = path.joinpath("data_storage", "gas.csv")

In [19]:
water_df = csv_to_dataframe(path_water, index_col=False)
gas_df = csv_to_dataframe(path_gas, index_col=False)
power_df = csv_to_dataframe(path_power, index_col=False)

In [20]:
water_df

,date_eom,date,days,w_id_meter,water_m3,w_cons,w_av_day
0,2011-06-30,2011-06-01,0.0,6.0,1.0,0.0,0.000000
1,2011-07-31,2011-07-12,41.0,6.0,10.0,9.0,0.219512
2,2011-08-31,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-09-30,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-10-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
191,2024-08-31,NaN,NaN,NaN,NaN,NaN,NaN
192,2024-09-30,NaN,NaN,NaN,NaN,NaN,NaN
193,2024-10-31,NaN,NaN,NaN,NaN,NaN,NaN
194,2024-11-30,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
water_df.head()

# remove "w_", or "g_" or "p_" from the column names
water_df.columns = water_df.columns.str.replace("w_", "")
gas_df.columns = gas_df.columns.str.replace("g_", "")
power_df.columns = power_df.columns.str.replace("p_", "")


In [25]:
water_df.head()

,date_eom,date,days,id_meter,water_m3,cons,av_day
0,2011-06-30,2011-06-01,0.0,6.0,1.0,0.0,0.000000
1,2011-07-31,2011-07-12,41.0,6.0,10.0,9.0,0.219512
2,2011-08-31,NaN,NaN,NaN,NaN,NaN,NaN
3,2011-09-30,NaN,NaN,NaN,NaN,NaN,NaN
4,2011-10-31,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# function to add id_meter
# def adding_data_meter_id(df):
#     df_process = df.copy()
#     # if water is part of
#     if "water" in df.columns:
#         df_process["id_meter"] = 1
#     # if df is gas
#     if "gas" in df.columns:
#         df_process["id_meter"] = 2
#     # if df is power
#     if "power" in df.columns:
#         df_process["id_meter"] = 3
#     # # if df is not water, gas or power
#     # if (
#     #     "water" not in df.columns
#     #     and "gas" not in df.columns
#     #     and "power" not in df.columns
#     # ):
#     #     print("The dataframe is not water, gas or power")
#     # if id_meter is 1 and date is lower than 2014-10-02
#     if df_process["id_meter"].iloc[0] == 1 & df_process["date"].iloc[0] < "2014-10-02":
#         df_process["id_meter"] = 4
#         # if id_meter is 1 and date is higher than 2014-10-02 and lower than 2020-10-03 then id_meter is 5
#     # elif (
#     #     df_process["id_meter"].iloc[0]
#     #     == 1 & df_process["date"].iloc[0]
#     #     > "2014-10-02" & df_process["date"].iloc[0]
#     #     < "2020-10-03"
#     # ):
#     #     df_process["id_meter"] = 5
#     # else:
#     #     df_process["id_meter"] = 6

#     # return the df processed
#     return df_process

In [ ]:
# adding_data_meter_id(water_df)

In [26]:
# Function to sort data based in the id_meter, and then in the date
def sort_data(df: pd.DataFrame) -> pd.DataFrame:
    df = df.sort_values(by=["id_meter", "date"])
    # if date empty fill with date_eom
    if "date_eom" in df.columns:
        df["date"] = df["date"].fillna(df["date_eom"])
        df = df.drop(columns=["date_eom"])
    # # drop empty rows in column 3
    # df = df.dropna(subset=[df.columns[3]])
    # convert date to datetime
    df["date"] = pd.to_datetime(df["date"])
    df = df.sort_values(by=["id_meter", "date"])
    # day_dif column as the difference between the date and the previous date
    df["days"] = df["date"].diff().dt.days
    # # replace 'days' by 'days_dif' and drop 'days_dif' column
    # df['days'] = df['days'].fillna(df['day_dif'])
    # df = df.drop(columns=['day_dif'])
    # if av_day fill with next value
    df["av_day"] = df["av_day"].bfill()

    return df


water_df = sort_data(water_df)
gas_df = sort_data(gas_df)
power_df = sort_data(power_df)

In [27]:
power_df

,date,days,id_meter,power_kwh,cons,av_day
39,2014-09-04,NaN,4.0,51419.3,0.0,0.000000
40,2014-09-12,8.0,4.0,NaN,NaN,2.841748
42,2014-11-28,77.0,4.0,NaN,NaN,2.841748
43,2014-12-16,18.0,4.0,51712.0,292.7,2.841748
44,2015-01-07,22.0,4.0,NaN,NaN,2.384264
...,...,...,...,...,...,...
191,2024-08-31,31.0,NaN,NaN,NaN,NaN
192,2024-09-30,30.0,NaN,NaN,NaN,NaN
193,2024-10-31,31.0,NaN,NaN,NaN,NaN
194,2024-11-30,30.0,NaN,NaN,NaN,NaN


In [28]:
# Function resampling independtly for each meter, droping the columns ['av_day', 'days'], and interpolating the data


def resample_data(df: pd.DataFrame, freq: str, name: str) -> pd.DataFrame:
    df_temp = df.copy()
    # split the data in as many dataframes as meters
    df_temp = df_temp.groupby("id_meter")
    # create a list of dataframes with the id_meter as key
    df_temp = [df_temp.get_group(x) for x in df_temp.groups]
    # convert the list of dataframes to independent dataframes
    for i in range(len(df_temp)):
        df_temp[i] = pd.DataFrame(df_temp[i])
    # resample each dataframe
    for i in range(len(df_temp)):
        # drop unnecessary columns ['av_day', 'days', 'water_m3', 'gas_m3', 'power_kwh', or 'cons']
        columns_drop = ["days", "av_day", "cons"]
        columns_drop = [col for col in columns_drop if col in df_temp[i].columns]
        df_temp[i] = df_temp[i].drop(columns=columns_drop)
        df_temp[i]["date"] = pd.to_datetime(df_temp[i]["date"])
        df_temp[i] = df_temp[i].set_index("date")
        df_temp[i] = df_temp[i].resample(freq).mean()
        df_temp[i] = df_temp[i].interpolate(method="time")
        df_temp[i] = df_temp[i].reset_index()
        # # drop duplicates
        # df_temp[i] = df_temp[i].drop_duplicates(subset=['date'])
        # create a new column [calc_cons] with the difference between the previous and the current value of the third column in the dataframe without using the name of the column
        df_temp[i]["calc_cons"] = df_temp[i].iloc[:, 2].diff()
        print("Dataframe shape: ", df_temp[i].shape)
    # merge the dataframes
    df_temp = pd.concat(df_temp)
    # sort the data by id_meter and date
    df_temp = df_temp.sort_values(by=["id_meter", "date"])
    # save the df in a csv file with the name 'resampled_data'+freq+df_name at the data_storage folder
    df_temp.to_csv(
        "../data_storage/resampled_data_" + freq + "_" + name + ".csv", index=False
    )
    return df_temp


interpolated_water_df = resample_data(water_df, "D", "water")
interpolated_gas_df = resample_data(gas_df, "D", "gas")
interpolated_power_df = resample_data(power_df, "D", "power")

Dataframe shape:  (1994, 4)
Dataframe shape:  (1566, 4)
Dataframe shape:  (1220, 4)
Dataframe shape:  (3559, 4)
Dataframe shape:  (1220, 4)
Dataframe shape:  (2374, 4)
Dataframe shape:  (1186, 4)
Dataframe shape:  (1220, 4)


In [29]:
print(interpolated_water_df)

           date  id_meter    water_m3  calc_cons
0    2014-09-04       2.0  114.418000        NaN
1    2014-09-05       2.0  114.742894   0.324894
2    2014-09-06       2.0  115.067788   0.324894
3    2014-09-07       2.0  115.392682   0.324894
4    2014-09-08       2.0  115.717576   0.324894
...         ...       ...         ...        ...
1215 2014-09-28       6.0  334.125000   0.218750
1216 2014-09-29       6.0  334.343750   0.218750
1217 2014-09-30       6.0  334.562500   0.218750
1218 2014-10-01       6.0  334.781250   0.218750
1219 2014-10-02       6.0  335.000000   0.218750

[4780 rows x 4 columns]


In [30]:
# function to group by month and year, aggregating by sum
def group_data_month(df: pd.DataFrame, name: str) -> pd.DataFrame:
    df_temp = df.copy()
    # add columns with year and month
    df_temp["year"] = df_temp["date"].dt.year
    df_temp["month"] = df_temp["date"].dt.month
    # drop date column
    df_temp = df_temp.drop(columns=["date"])
    # group by id_meter, year and month, aggregating by sum
    df_temp = df_temp.groupby(
        [
            "year",
            "month",
            "id_meter",
        ]
    ).sum()
    # merge the columns year and month and assign the last day of the month
    df_temp = df_temp.reset_index()
    df_temp["date"] = pd.to_datetime(df_temp[["year", "month"]].assign(day=1))
    df_temp["date"] = df_temp["date"] + pd.offsets.MonthEnd(0)
    # drop year and month columns
    df_temp = df_temp.drop(columns=["year", "month"])
    # set the date as index
    df_temp = df_temp.set_index("date")
    # reset index
    df_temp = df_temp.reset_index()
    # sort the data by id_meter and date
    df_temp = df_temp.sort_values(by=["id_meter", "date"])
    # save the df in a csv file with the name 'grouped_data_month_'+name at the data_storage folder
    df_temp.to_csv("../data_storage/grouped_data_month_" + name + ".csv", index=False)
    return df_temp


grouped_water_df = group_data_month(interpolated_water_df, "water")
grouped_gas_df = group_data_month(interpolated_gas_df, "gas")
grouped_power_df = group_data_month(interpolated_power_df, "power")